# Introduction to Reinforcement Learning with NVIDIA Jetson TX2

In this session, you will use a branch of machine learning called **reinforcement learning** to teach a robot to play a game.

## The Game

The setup has four LEDs.  We enumerate the LEDs starting with zero, so that the yellow LED is at position `0`, the red LED is at position `1`, and so on.  Each LED is connected to a button that is used to turn it off.  

![LEDs](images/LEDs.png)

The game round begins when **one** of the four LEDs is turned on, and the robotic arm starts at a position hovering over one of the four buttons.  The score always starts at zero.

At each point in the game, the robot has three potential movements (or **actions**) at its disposal.  It can:
- `0` - move **L**eft one position,
- `1` - stay at the current position and **P**ush, or
- `2` - move **R**ight one position.

After the robot chooses an action, its score is deducted one point.  While most actions have intuitive effects, there are a few special cases that are worth mentioning:
- The lit LED is turned off when the robot pushes its corresponding button, but pushing buttons connected to unlit LEDs has no effect (i.e., they are never turned on by the robot).  
- If the robotic arm is hovering over the leftmost button at location `0` and decides to move left, we imagine that the arm hits an imaginary wall, and the arm stays where it is.  Likewise, if the arm is hovering over the button at position `3` and decides to move right, then at the next point in the game, the arm will have maintained its position at location `3` (the rightmost location in the line).

The round ends when the robot pushes the correct button to turn off the lit LED.  Your goal in this notebook is to implement an algorithm that can learn from gameplay the optimal strategy to attain the highest (or least negative) score at each round.  To accomplish this, your robot will need to learn how to turn off the lit LED in as few movements as possible.  

## Motivating Reinforcement Learning

You may have noticed that the winning game strategy is very straightforward to hard code.  With this in mind, if you are new to RL, you might wonder why an RL technique would not be overkill.

And the fact is that in the field of RL, it is very common to use simple games with well-defined rules **to build intuition** for how to design algorithms to best accomplish more complex tasks.  

In the RL setting, we assume that the robot does not have the domain knowledge to know what "left", "push", or "right" means.  The robot only knows that it has three possible actions, and it needs to figure out how to select from these actions to consistently attain the highest possible score.  

You can think of the robot as a computer player with full access to a keyboard that only contains three keys, where you further assume the player does not know any of the game controls.  Despite this missing information, the player must nonetheless learn how to beat the game.  

And, similar to how you can imagine the human player would learn, the robot will closely watch its score in the game to gauge how well it is doing.  If it is not performing well, it will amend its strategy to do better.  

The algorithm that you will use to determine the best gameplay strategy is remarkably intuitive.  The robot will learn primarily through trial-and-error, while using its score as a feedback mechanism to hone its strategy.  Its initial behavior will be incredibly random, as it tries out many different moves (or actions) in the game, to see how its score is affected.  The main idea behind the algorithm is to train the robot to leverage its gameplay experience to gradually obtain a well-informed strategy that attains a relatively high score.  

For now, in the next section, we will examine how the robot performs, if it chooses random actions at each time step.

## Investigate Random Behavior

We have written a simple simulator that you can use to see how the robot should perform, if it selects random actions for the entirety of the game.  Of course, your robot will learn to perform much better!

Run the code cell below to have the robot play the game for 2 separate rounds.  When parsing the output, remember that the starting conditions of the game are random!

Each round has corresponding output that looks somewhat like the snippet below:
```
Starting Round 1 ...
 LED: 0    | Arm: 1 | Score:    0 
 Action: P | Arm: 1 | Score:   -1 
 Action: L | Arm: 0 | Score:   -2 
 Action: P | Arm: 0 | Score:   -3 
FINAL SCORE:  -3
-----------------------------------
```

In the sample snippet above:
- When the game round was initiated, the lit LED was at position `0`, and the arm was at position `1`.  The game score at the beginning of a round is always zero. 
- The robot's first choice of action was to **P**ush in the current location; as a result, its score was deducted one point, and the LEDs were unaffected (i.e., the LED at position `0` remains lit, while all of the other LEDs are unlit).  
- The robot's next choice was to move **L**eft, so it moved to position `0` and the score was deducted another point.
- Then, the robot decided to **P**ush in location `0`, and the game score decreased to `-3`.  At this point, the game ended, because the final action turned off the lit LED. 
- In this case, the final score received at the end of the game was `-1` + `-1` + `-1` = `-3`.

Take the time now to understand the `JetsonEnv` class in **jetson_env.py**.  Note that the simulation encodes each of the possible actions as an integer (one of `0`, `1`, or `2`), and to get the corresponding more interpretable action label (`L`, `P`, or `R`), we use the `decipher_action` function below. 

Later in this notebook, you will use the `JetsonEnv` class to simulate games to teach your robot!

In [1]:
from jetson_env import JetsonEnv

# use a Python dictionary to decode the actions
action_dict = {0: 'L', 1: 'P', 2: 'R'}
def decipher_action(a):
    return action_dict[a]

# create a new environment
env = JetsonEnv()

# interact with the environment
for i_round in range(1, 3):
    print('Starting Round %d ...' % i_round)
    # reset the lit LED, arm position, and score
    led, arm = env.reset()
    score = 0
    print(' LED: %d    | Arm: %d | Score: %4d ' % (led, arm, score))
    while True:
        action = env.get_random_action() # select a random action
        arm, points, done = env.step(action)
        score += points
        print(' Action: %s | Arm: %d | Score: %4d ' % (decipher_action(action), arm, score))
        if done:
            print('FINAL SCORE: ', score)
            print('-'*35)
            break

Starting Round 1 ...
 LED: 2    | Arm: 0 | Score:    0 
 Action: R | Arm: 1 | Score:   -1 
 Action: L | Arm: 0 | Score:   -2 
 Action: L | Arm: 0 | Score:   -3 
 Action: P | Arm: 0 | Score:   -4 
 Action: R | Arm: 1 | Score:   -5 
 Action: L | Arm: 0 | Score:   -6 
 Action: P | Arm: 0 | Score:   -7 
 Action: P | Arm: 0 | Score:   -8 
 Action: P | Arm: 0 | Score:   -9 
 Action: P | Arm: 0 | Score:  -10 
 Action: L | Arm: 0 | Score:  -11 
 Action: P | Arm: 0 | Score:  -12 
 Action: R | Arm: 1 | Score:  -13 
 Action: P | Arm: 1 | Score:  -14 
 Action: P | Arm: 1 | Score:  -15 
FINAL SCORE:  -15
-----------------------------------
Starting Round 2 ...
 LED: 0    | Arm: 2 | Score:    0 
 Action: R | Arm: 3 | Score:   -1 
 Action: L | Arm: 2 | Score:   -2 
 Action: L | Arm: 1 | Score:   -3 
 Action: L | Arm: 0 | Score:   -4 
 Action: L | Arm: 0 | Score:   -5 
 Action: L | Arm: 0 | Score:   -6 
 Action: R | Arm: 1 | Score:   -7 
 Action: L | Arm: 0 | Score:   -8 
 Action: P | Arm: 0 | Score: 

## States, Actions, and Optimal Policies

As discovered above, there are **three possible actions**, corresponding to:
- `0` - moving **L**eft, 
- `1` - staying and **P**ushing in the current position, and
- `2` - moving **R**ight.

The **total number of possible game states is $4^2 = 16$**, where there is a state for each possible combination of arm position and lit LED position. 

To avoid having to deal with two different numbers when referencing the state, we define the `get_state` function below that maps each possible combination of arm position (`arm`) and lit LED position (`led`) to an integer from `0` to `15`, which we refer to as the corresponding state (`state`).  

In your upcoming implementation, the state should always be encoded as a number from `0` to `15`, but you can get the corresponding arm position (`arm`) and lit LED position (`led`) by passing the state (`state`) into the `get_led_and_arm` function.

In [2]:
def get_led_and_arm(state, nLED=4):
    arm = state % nLED
    led = int(((state-arm))/nLED)
    return led, arm

def get_state(led, arm, nLED=4):
    state = led*nLED + arm
    return state

for state in range(16):
    led, arm = get_led_and_arm(state)
    print('LED: %d | Arm: %d | State: %2d ' % (led, arm, state))

LED: 0 | Arm: 0 | State:  0 
LED: 0 | Arm: 1 | State:  1 
LED: 0 | Arm: 2 | State:  2 
LED: 0 | Arm: 3 | State:  3 
LED: 1 | Arm: 0 | State:  4 
LED: 1 | Arm: 1 | State:  5 
LED: 1 | Arm: 2 | State:  6 
LED: 1 | Arm: 3 | State:  7 
LED: 2 | Arm: 0 | State:  8 
LED: 2 | Arm: 1 | State:  9 
LED: 2 | Arm: 2 | State: 10 
LED: 2 | Arm: 3 | State: 11 
LED: 3 | Arm: 0 | State: 12 
LED: 3 | Arm: 1 | State: 13 
LED: 3 | Arm: 2 | State: 14 
LED: 3 | Arm: 3 | State: 15 


The goal of your robot is to find - for each possible game state - the best action that the robot should take from that state, towards its goal of maximizing the game score.  We will think of this as a lookup table that the robot can consult when selecting actions, and we refer to it as an **optimal policy**.

For instance, consider the case that the game starts in state `0`.  This state corresponds to arm position `0` and lit LED position `0`.  In this case, the robot should select action **P**ush, to end the game with a final score of `-1` immediately after.

Likewise, state `1` corresponds to arm position `1` and lit LED position `0`.  In this case, the robot should decide to move **L**eft as the best initial move.  (This way, the robot can select to **P**ush at the next step and end the game with a best final score of `-2`.)

Take the time now to look at the printed optimal policy below.  Check to make sure that you can see why these actions are optimal, in the context of their corresponding game states!

```
LED: 0 | Arm: 0 | State:  0 | Best Action: P
LED: 0 | Arm: 1 | State:  1 | Best Action: L
LED: 0 | Arm: 2 | State:  2 | Best Action: L
LED: 0 | Arm: 3 | State:  3 | Best Action: L
LED: 1 | Arm: 0 | State:  4 | Best Action: R
LED: 1 | Arm: 1 | State:  5 | Best Action: P
LED: 1 | Arm: 2 | State:  6 | Best Action: L
LED: 1 | Arm: 3 | State:  7 | Best Action: L
LED: 2 | Arm: 0 | State:  8 | Best Action: R
LED: 2 | Arm: 1 | State:  9 | Best Action: R
LED: 2 | Arm: 2 | State: 10 | Best Action: P
LED: 2 | Arm: 3 | State: 11 | Best Action: L
LED: 3 | Arm: 0 | State: 12 | Best Action: R
LED: 3 | Arm: 1 | State: 13 | Best Action: R
LED: 3 | Arm: 2 | State: 14 | Best Action: R
LED: 3 | Arm: 3 | State: 15 | Best Action: P
```

Once the robot has determined the optimal policy, it can reference it when playing the game, in order to consistently attain the highest possible score.  For instance, if the robot is presented with a situation in the game where the lit LED is at position 2, and its arm is at position 1, it need only find the line corresponding to state 9 in the table, where it will see that the best action is to go left.

Next, you will implement a method known as **Monte Carlo with Exploring Starts** to guide your robot to obtain this optimal policy.

## Monte Carlo with Exploring Starts

As part of this method, the robot will maintain a numpy array `Q` with `16` rows and `3` columns.  
> The entry in the `s`-th row and `a`-th column (`Q[s,a]`) contains the robot's *estimate* for the highest score it can possibly obtain in the game, if the game started in state `s`, and the robot selected action `a` for its first move.  

For the first several games, the estimates in the array will be wildly inaccurate; but, the more experience the robot gets with gameplay, the more it is able to refine these estimates.

It is relatively straightforward to determine what these estimates _should_ be, _if_ they are completely accurate.  For instance, consider `Q[3,1]`.  This value corresponds to state `3`, which corresponds to the case where the game starts with the lit LED at position `0`, and the robotic arm is at position `3` (*... please verify this in the section above!*).  If the robot selects action `1` as its first move in the game, then this corresponds to pushing in the current location, which results in a score of `-1`.  Then, at this point, if the agent plays optimally, it can obtain a final score of **at most** `-5`, obtained by selecting actions `0`, `0`, `0`, and `1` to end the game in four more moves.  The agent will try to estimate this value (`Q[3,1]=-5`), along with all other values in the array, from gameplay.  And - amazingly - it will refine these estimates while simultaneously trying to determine the optimal policy.  That is, it does not need to know the optimal policy *a priori* before forming these estimates!

This array is deeply connected to the optimal policy.  What's particularly worth noting is that in the event that we have a perfect estimate of all of the values in `Q`, we can quickly use it to obtain the optimal policy.  To see this, suppose for now that the robot has a perfect estimate of all of the entries in the `Q` array.  Then, it can get the corresponding optimal action corresponding to any state `s` simply by looking at the `s`-th row in `Q`, or `Q[s]`.  As an example, if `s=3`, then `Q[3]=[-4, -5, -5]`.  
> Then, to get the optimal action corresponding to state `s`, we need only select the action corresponding to the index that maximizes `Q[s]`.  

In the case that `s=3`, we see that `-4` is the largest entry in `Q[s]` (which appears at index `0`), and so the optimal action is action `0` (which corresponds to action left).  Take the time now to check that this lines up with both your intuition and the optimal policy above!  To understand more generally why this is the case, note that each entry in `Q[s]` contains the final score if the robot follows the optimal policy, for all moves except (potentially) the first.  Then, one of the three available initial moves must be optimal, and it makes sense that the optimal move will correspond to the action (or index) that yields the largest value in `Q[s]`.  This fact will come in very useful in the next section, when we talk about how to implement the method in more detail.

### Implementation Details

The algorithm begins with an initial guess for the values in `Q`.  In this session, we initialize `Q[s,a]=-5` for each `s` and `a`.  Of course, this initial estimate isn't so great, and we provide this bad initial estimate so that the robot can learn for itself how to improve it.  We don't want to give the robot too much of a head start!

Then, as the robot plays the game, the estimates in `Q` will be gradually improved.  You'll have the chance to implement this soon.

When playing the game, the robot will use the (potentially inaccurate) estimates in `Q` to select actions.  With every game round, the robot will evaluate the current game state, and then choose the action by selecting the index that maximizes `Q[s]`.  Of course, as `Q` gets more and more accurate, the robot will select better and better actions!  That is, its strategy for selecting actions will gradually start to resemble the optimal policy.

You can see this implementation below in the `play_round` function.  Note that the function takes the estimates in `Q` as input.  It returns a list of `(state, action)` tuples, detailing how the game evolved. 

The output of the code cell prints how the game evolved for three different rounds.

Each round has corresponding output that looks somewhat like the snippet below:
```
Round 1: [(9, 2), (10, 1)] | Score: -2 
```
In the sample snippet above, the game began in state `9`, and the robot's initial action was `2`.  This changed the game to state `10`, and the robot selected action `1` to end the game.  The robot's final score was `(-1)+(-1)=-2`.

Note that since the robot is always deducted a point after each choice of action, we can obtain the robot's game score from the length of the `game_round` list that is returned by the `play_round` function.

In [3]:
import numpy as np
import random

def play_round(env, Q=None):
    if Q is None: # default value for Q if not provided
        Q = -5*np.ones((env.nS, env.nA), dtype=float)
    # start the round
    game_round = []
    led, arm = env.reset()                   # reset the game
    state = get_state(led, arm, env.nLED)    # initial state

    # select a random action
    action = env.get_random_action()            # select initial random action 
    arm, _, done = env.step(action)             # take initial action
    game_round.append((state, action))          # save initial state and initial action

    # finish the round
    while not done:
        state = get_state(led, arm, env.nLED)       # get next game state
        best_actions = np.argwhere(Q[state] == np.amax(Q[state])).flatten() # all actions that maximize Q[state]
        action = random.choice(best_actions)        # select action from set of best actions
        arm, _, done = env.step(action)             # take action
        game_round.append((state, action))          # save state and action
    
    return game_round

# play the game for three separate rounds
for i in range(1,4):
    game_round = play_round(env)
    print('Round %d: %s | Score: %d' % (i, game_round, -len(game_round)))

Round 1: [(2, 0), (1, 0), (0, 2), (1, 1), (1, 2), (2, 2), (3, 0), (2, 2), (3, 2), (3, 2), (3, 0), (2, 1), (2, 0), (1, 2), (2, 2)] | Score: -15
Round 2: [(15, 2), (15, 2), (15, 0), (14, 1), (14, 2), (15, 1)] | Score: -6
Round 3: [(5, 0), (4, 0), (4, 0), (4, 0), (4, 2), (5, 0), (4, 2), (5, 2), (6, 2), (7, 2), (7, 2), (7, 2), (7, 0), (6, 0), (5, 0)] | Score: -15


You will implement the **Monte Carlo with Exploring Starts** method in the `monte_carlo` function in the code cell below.  As described above, we begin by initializing `Q[s,a]=-5` for each `s` and `a`.  Then, the robot plays a single round using the `play_round` function, and the results of the round (in `game_round`) are used to update the estimates in `Q`.  The loop continues for a total of `num_rounds` game rounds, where the agent updates the estimates in `Q` between rounds.  Although the robot will initially be quite bad at choosing game moves, it will demonstrate a high level of skill later in gameplay!

Some of this code has been provided for you, and your task is to write the code to update the values in `Q`.  

Don't worry about this just yet - for now, run the code cell below to verify that the code works.  *And so here's the main idea behind how the algorithm should operate ...*

In [4]:
import sys

def monte_carlo(env, num_rounds=100):
    nS = env.nS        # number of states
    nA = env.nA        # number of actions
    
    # initialize empty arrays
    Q = -5*np.ones((nS, nA), dtype=float)
    scores_sum = np.zeros((nS, nA), dtype=float)
    scores_count = np.zeros((nS, nA), dtype=float)
    
    # loop over game rounds
    for i_round in range(1, num_rounds+1):
        
        # monitor progress
        print("\rGame Round {}/{}.".format(i_round, num_rounds), end="")
        sys.stdout.flush()
        
        # play game round
        game_round = play_round(env, Q)
        
        # TODO: use game round to update Q
        for state, action in set(game_round):
            first_idx = min([i for i,x in enumerate(game_round) if x==(state,action)])
            scores_sum[state][action] += -len(game_round[first_idx:])
            scores_count[state][action] += 1
            Q[state][action] = scores_sum[state][action]/scores_count[state][action]
            
    return Q

# run the algorithm
Q = monte_carlo(env)

# print the learned policy
print('\n\nEstimated Optimal Policy:')
for state in range(Q.shape[0]):
    led, arm = get_led_and_arm(state, env.nLED)
    print('LED: %d | Arm: %d | State: %2d | Best Action: %s' % (led, arm, state, decipher_action(np.argmax(Q[state]))))

# cleaner printing of learned policy
print('\nCorrect Entries: %d/16' % sum(np.argmax(Q, axis=1) == [1, 0, 0, 0, 2, 1, 0, 0, 2, 2, 1, 0, 2, 2, 2, 1]))

Game Round 100/100.

Estimated Optimal Policy:
LED: 0 | Arm: 0 | State:  0 | Best Action: P
LED: 0 | Arm: 1 | State:  1 | Best Action: L
LED: 0 | Arm: 2 | State:  2 | Best Action: L
LED: 0 | Arm: 3 | State:  3 | Best Action: L
LED: 1 | Arm: 0 | State:  4 | Best Action: R
LED: 1 | Arm: 1 | State:  5 | Best Action: P
LED: 1 | Arm: 2 | State:  6 | Best Action: L
LED: 1 | Arm: 3 | State:  7 | Best Action: L
LED: 2 | Arm: 0 | State:  8 | Best Action: R
LED: 2 | Arm: 1 | State:  9 | Best Action: R
LED: 2 | Arm: 2 | State: 10 | Best Action: P
LED: 2 | Arm: 3 | State: 11 | Best Action: L
LED: 3 | Arm: 0 | State: 12 | Best Action: R
LED: 3 | Arm: 1 | State: 13 | Best Action: R
LED: 3 | Arm: 2 | State: 14 | Best Action: R
LED: 3 | Arm: 3 | State: 15 | Best Action: P

Correct Entries: 16/16


## Not Using / Learn More about RL

but it will prove useful to define some additional terminology.

In the field of RL, we refer to the equipment that controls the robot as the decision-making **agent**.  The agent must consider the **state** of the game when selecting **actions** that influence gameplay. 

We think of each game round as broken into discrete time steps.  At the start of a round, the agent observes the initial state of the game, which we denote by $s_0$.  Using the context provided by the state, the agent has to choose an appropriate action $a_0$ in response.  This action likely changes the state of the game, and we denote the following game state as $s_1$.

After the agent chooses an action, the score of the game is deducted by one point.  This negative reinforcement is used to signal to the agent that it should try to end the game as quickly as possible.  

We can translate this change in game score to the traditional RL framework by saying that the agent received a **reward** of -1.  In particular, we can say the agent received a reward $r_1=-1$ as a result of selecting action $a_0$ after observing state $s_0$.

We refer to a complete game round, from start to finish, as an **episode**.  Each episode evolves as a sequence of states, actions, and rewards 

$$
(s_0, a_0, r_1, \ldots, s_t, a_t, r_{t+1}, \ldots, s_{T-1}, a_{T-1}, r_T, s_T),
$$ 

where $T$ is the final time step, and $s_T$ is the final game state (where the LED has been turned off by the robotic arm).  The final score of the agent can be identified as the sum 
$$
\sum_{t=1}^Tr_t = r_1 + \ldots + r_T,
$$
which we also refer to as the **cumulative reward** collected by the agent over the course of the episode.  Note that this cumulative reward is equivalent to the robot's final score at the end of the game.

Our goal is to teach the agent to develop a game-playing strategy to maximize cumulative reward, and you will learn how to design an algorithm to accomplish this soon.  